In [25]:
import pandas as pd
import numpy as np

# Đường dẫn đến file feather cần đọc
file_path = 'D:/GQUIC_small/Test/GQUIC_test_128.feather'

# Đọc file feather
df = pd.read_feather(file_path)

In [26]:
label_counts = df['Label'].value_counts()
label_counts/20

4    903.0
0    727.0
2    682.0
1    522.0
3    458.0
Name: Label, dtype: float64

In [2]:
# Chọn cột '0' đến '127' cho mỗi flow_id và chuyển thành một danh sách các ma trận
matrices = []
labels = []
for flow_id, group in df.groupby('flow_id'):
    matrix = group.loc[:, '0':'63'].values
    label = group['Label'].values[0] 
    matrices.append(matrix)
    labels.append(label)

# Chuyển danh sách ma trận thành mảng NumPy
X = np.array(matrices)
X = X / 255
y = np.array(labels)

In [3]:
# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 64, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(4, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 64, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 64, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 32, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 32, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 32, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 32, 64)        36928     
                                                        

In [4]:
model.fit(X, y, epochs=10, validation_data=(X, y))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "D:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "D:\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "D:\anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "D:\anaconda\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "D:\anaconda\Lib\asyncio\base_events.py", line 1922, in _run_once
      handle._run()
    File "D:\ana

In [ ]:
# Đường dẫn đến file feather cần đọc
file_path_test = 'D:/GQUIC_small/dataset1/Test/test_Transfer_64.feather'

# Đọc file feather
df_test = pd.read_feather(file_path)

In [ ]:
# Chọn cột '0' đến '127' cho mỗi flow_id và chuyển thành một danh sách các ma trận
matrices_test = []
labels_test = []
for flow_id, group in df.groupby('flow_id'):
    matrix = group.loc[:, '0':'63'].values
    label = group['Label'].values[0] 
    matrices_test.append(matrix)
    labels_test.append(label)

# Chuyển danh sách ma trận thành mảng NumPy
X_test = np.array(matrices_test)
X_test = X_test / 255
y_test = np.array(labels_test)

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy * 100)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)


In [ ]:
# Đếm số lần xuất hiện của mỗi giá trị trong y_pred_classes
counts = np.bincount(y_pred_classes)

# Hiển thị kết quả
for class_idx, count in enumerate(counts):
    print(f"Số lần xuất hiện của lớp {class_idx}: {count}")